In [1]:
import urllib.request as req
import json
import csv
import datetime

In [4]:
baseurl = "https://api.github.com/repos/%s/%s/issues/%s/comments?page=%s"

owner = "torvalds"
repo = "linux"
issue = 17
pages = [1, 2, 3, 4]

def get_comments_page(owner, repo, issue, page):
    issue = str(issue)
    page = str(page)
    return req.urlopen(
        baseurl % (owner, repo, issue, page)
    )

In [5]:
# Fetch data
response = []
for p in pages:
    page = get_comments_page(owner, repo, issue, p)
    response.append(page.read())

In [6]:
response = [json.loads(response[i].decode('utf-8')) for i in range(len(response))]

In [8]:
cutoff_date = datetime.datetime.strptime("2012-05-13", "%Y-%m-%d")

csv_str = ""
for page in response:
    for el in page:
        datestr = el['created_at']
        dt = datetime.datetime.strptime(datestr, "%Y-%m-%dT%H:%M:%SZ")
        if dt < cutoff_date:
            login = el['user']['login']
            body = el['body'].replace("\n", " ").replace("\r", " ")
            csv_str+=login+"\t"+body+"\n"

with open("github-comments.csv", "w") as csvfile:
    csvfile.write(csv_str)

In [9]:
csv.register_dialect(
    'github-dialect',
    delimiter="\t",
    quoting=csv.QUOTE_NONE)
with open("github-comments.csv", "r") as f:
    data = csv.reader(f, 'github-dialect')
    comments = [row for row in data]